In [5]:
!pip install qiskit qiskit_aer matplotlib pylatexenc --quiet

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

def swap_registers(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

def qft_circuit(n):
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

def inverse_qft_circuit(n):
    qc = QuantumCircuit(n)
    swap_registers(qc, n)
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / 2**(j - k), k, j)
        qc.h(j)
    qc.name = "Inverse QFT"
    return qc

def run_qft(n):
    qc = qft_circuit(n)
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()
    statevector = result.data(0)["statevector"]
    print(f"\nQFT circuit for {n} qubits:")
    display(qc.draw('mpl'))
    plot_bloch_multivector(statevector)
    plt.show()

def phase_estimation_example():
    phase = 0.25
    n_count = 3
    qc = QuantumCircuit(n_count + 1, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(n_count)
    for q in range(n_count):
        qc.cp(2 * np.pi * phase * (2**q), q, n_count)
    iqft = inverse_qft_circuit(n_count)
    qc.append(iqft.to_gate(), range(n_count))
    qc.measure(range(n_count), range(n_count))
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()
    print("\nPhase Estimation Circuit:")
    display(qc.draw('mpl'))
    plot_histogram(counts)
    plt.show()
    return counts

